In [19]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser

from flask import Flask, render_template, redirect
from webdriver_manager.chrome import ChromeDriverManager

##from flask_pymongo import PyMongo
import time
import pandas as pd

In [20]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\lily\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


# NASA Mars News
<li>Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [21]:
# Visit NASA MARS News Site that to be scraped
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)
time.sleep(1)

# Scrape page into Soup
news_html = browser.html
news_soup = bs(news_html, "html.parser")

## Debug: examine the result
# print(news_soup.prettify())

In [22]:
list_text_all = news_soup.find_all('div', class_='list_text')

# for list_text in list_text_all:
#     print(list_text)


In [23]:
## scrape the first result from the ResultSet (list_text_all):
content_title = list_text_all[0].find('div', class_ = 'content_title')
news_title = content_title.text.strip()

news_paragraph = list_text_all[0].find('div', class_ = 'article_teaser_body')
news_p = news_paragraph.text.strip()

print("new title: ", news_title)
print("news_p: ", news_p)

new title:  NASA's Ingenuity Mars Helicopter Completes First One-Way Trip
news_p:  The Red Planet rotorcraft headed south in support of furthering research into the potential use of aerial scouts on Mars in the future.


# JPL Mars Space Images - Featured Image
  1) Visit the url for the Featured Space Image site here.   
  2) Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.  
  3) Make sure to find the image url to the full size .jpg image.   
  4) Make sure to save a complete url string for this image.  

In [24]:
nasa_url = 'https://www.jpl.nasa.gov'
image_url = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
# img_download_url = 'https://www.jpl.nasa.gov/images/baldet-crater-false-color-2'

browser.visit(image_url)
time.sleep(3)

# Scrape page into Soup
img_html = browser.html

## Parse HTML
img_soup = bs(img_html, "html.parser")

# detail_url = browser.links.find_by_partial_href('/images')
# time.sleep(2)
# print(detail_url[0].text)

## retrieve the list of relative URL for image detail page, where the download button can be found to a full-size image
## Found this HTML via Inspecting page source: <a href="/images/black-and-white-view-of-ingenuitys-fourth-flight" class="group  cursor-pointer block">
image_detail_page_all = img_soup.select('a[href^="/images/"]')

## Debug: Check a list of relative url to the detailed image page
# for i in image_detail_page_all:
#     print(i['href'])
    
## Pick an image's relative URL, contruct a full url by prefixing the nasa_url.
image_detail_page_url = nasa_url + image_detail_page_all[1]['href']

print(f"image_detail_url = {image_detail_page_url}")


image_detail_url = https://www.jpl.nasa.gov/images/ingenuitys-successful-fifth-flight


In [25]:
## Visit the image detail page
browser.visit(image_detail_page_url)
time.sleep(3)

# Scrape page into Soup
img_detail_html = browser.html

## Parse HTML to find the URL for the full featured image: <a data-v-0526c969="" href="https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24647.jpg" class="BaseButton text-contrast-none w-full mb-5 -primary -compact inline-block"><span class="label block">Download JPG </span></a>
img_detail_soup = bs(img_detail_html, "html.parser")

## find the "a" tag with href contains "original_images", save the URL
featured_full_image_url = img_detail_soup.select('a[href*="original_images"]')[0]['href']

print(f"The featured_full_image_url = {featured_full_image_url}")

The featured_full_image_url = https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24647.jpg


# Mars Facts
1) Visit the Mars Facts webpage here (https://galaxyfacts-mars.com/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.   
2) Use Pandas to convert the data to a HTML table string.  

In [26]:
  # Visit Mars Facts page for Mars' facts
mars_facts_url = "https://galaxyfacts-mars.com/"
browser.visit(mars_facts_url)
html = browser.html

In [27]:
# Use Pandas to scrape the fact table about Mars
mars_facts_tables = pd.read_html(mars_facts_url)
mars_facts_tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [28]:
mars_facts_df = mars_facts_tables[0]
mars_facts_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [29]:
# Rename columns
mars_facts_df.columns = ['Description','Mars', 'Earth']

# Reset Index to be description
mars_facts_df.set_index('Description', inplace=True)

mars_facts_df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [30]:
# Use Pandas to convert the data to a HTML table string
mars_facts_df.to_html('mars_fact_table.html')

# Mars Hemispheres
1) Visit the USGS Astrogeology site here (https://marshemispheres.com/) to obtain high resolution images for each of Mars's hemispheres.   
2) You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.  
3) Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name.  
4) Use a Python dictionary to store the data using the keys img_url and title.  
5) Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.  

In [31]:
hemi_url = 'https://marshemispheres.com/'
# img_download_url = 'https://www.jpl.nasa.gov/images/baldet-crater-false-color-2'

browser.visit(hemi_url)
time.sleep(3)

# Scrape page into Soup
html_hemi = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_hemi, 'html.parser')

# Retreive all 4 items that contain mars hemispheres information
items_all = soup.find_all('div', class_='item')
# print(items_all)

In [32]:

# Create an empty list for hemisphere urls 
hemi_image_urls = []

# Loop through all 4 items
for item in items_all: 
    # Store title
    title = item.find('h3').text
    
    # save the partial image URL
    partial_img_url = item.find('a', class_='itemLink product-item')['href']
    
    # Go to the image link
    browser.visit(hemi_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for each hemisphere website 
    soup = bs( partial_img_html, 'html.parser')
    
    # Retrieve the full image source from html tag "<img class="wide-image" src="/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg">"
    full_img_url = hemi_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a dict list 
    hemi_image_urls.append({"title" : title, "full_img_url" : full_img_url})
    
# Display hemisphere_image_urls
hemi_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'full_img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'full_img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'full_img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'full_img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]